In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=7d2186486424cb9e8fd63cc8f2632c586a5180a230bcc6c7f1eb5d7045210da1
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('cruise').getOrCreate()

In [4]:
df = spark.read.csv("/content/7th_prog_dataset.csv",
header=True, inferSchema=True)

In [5]:
df.show(10)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [6]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [7]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [8]:
from pyspark.ml.feature import StringIndexer

In [9]:
indexer=StringIndexer(inputCol='Cruise_line',outputCol='cruise_cat')

In [10]:
indexed=indexer.fit(df).transform(df)

In [11]:
for item in indexed.head(5):
  print(item)
print('\n')

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)
Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)
Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0)
Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0)
Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, cruise_cat=1.0)




In [12]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [13]:
assembler=VectorAssembler(inputCols=['Age',
'Tonnage',
'passengers',
'length',
'cabins',
'passenger_density','crew',
'cruise_cat'],outputCol='features')

In [14]:
output=assembler.transform(indexed)

In [15]:
output.select('features','crew').show(5)

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
+--------------------+----+
only showing top 5 rows



In [16]:
final_data=output.select('features','crew')

In [17]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [18]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               107|
|   mean| 7.701682242990664|
| stddev|3.5491725014780204|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [19]:
test_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               51|
|   mean|7.988235294117646|
| stddev|3.432216022270365|
|    min|              0.6|
|    max|             19.1|
+-------+-----------------+



In [20]:
from pyspark.ml.regression import LinearRegression

In [21]:
ship_lr=LinearRegression(featuresCol='features',labelCol='crew')

In [22]:
trained_ship_model=ship_lr.fit(train_data)

In [23]:
ship_results=trained_ship_model.evaluate(train_data)
print('Rsquared Error :',ship_results.r2)

Rsquared Error : 1.0


In [24]:
unlabeled_data=test_data.select('features')

In [25]:
unlabeled_data.show(5)

+--------------------+
|            features|
+--------------------+
|[5.0,86.0,21.04,9...|
|[5.0,122.0,28.5,1...|
|[5.0,160.0,36.34,...|
|[6.0,30.276999999...|
|[6.0,110.23899999...|
+--------------------+
only showing top 5 rows



In [26]:
predictions=trained_ship_model.transform(unlabeled_data)

In [27]:
predictions.show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[5.0,86.0,21.04,9...|       8.0|
|[5.0,122.0,28.5,1...|       6.7|
|[5.0,160.0,36.34,...|      13.6|
|[6.0,30.276999999...|      3.55|
|[6.0,110.23899999...|      11.5|
|[6.0,113.0,37.82,...|      12.0|
|[7.0,116.0,31.0,9...|      12.0|
|[8.0,77.499,19.5,...|       9.0|
|[8.0,110.0,29.74,...|      11.6|
|[9.0,85.0,19.68,9...|      8.69|
|[9.0,105.0,27.2,8...|     10.68|
|[9.0,110.0,29.74,...|      11.6|
|[9.0,113.0,26.74,...|     12.38|
|[10.0,68.0,10.8,7...|      6.36|
|[10.0,77.0,20.16,...|       9.0|
|[10.0,86.0,21.14,...|       9.2|
|[10.0,90.09,25.01...|      8.58|
|[11.0,86.0,21.24,...|       9.3|
|[11.0,90.0,22.4,9...|      11.0|
|[11.0,90.09,25.01...|      8.48|
+--------------------+----------+
only showing top 20 rows

